In [39]:
from abc import ABC, abstractmethod

In [40]:
import random
class FakeLLM(Runnable):
    def __init__(self):
        print('LLM Created')
    
    def invoke(self, prompt):
        response_list= [
            'Dhaka is the capital of Bangladesh.',
            'BPL is the top professional football league in Bangladesh.',
            'AI stands for Artificial Intelligence.',
        ]
        return {'response':random.choice(response_list)}
        
# keep the predict method for compatibility
    def predict(self, prompt):
        response_list= [
            'Dhaka is the capital of Bangladesh.',
            'BPL is the top professional football league in Bangladesh.',
            'AI stands for Artificial Intelligence.',
        ]
        return {'response':random.choice(response_list)}

In [41]:
class FakePromptTemplate(Runnable):
    def __init__(self, template,input_variables):
        self.template = template
        self.input_variables = input_variables
        print('Prompt Template Created')

    def invoke(self, input_dict):
        return self.template.format(**input_dict)  
         
    # keep the format method for compatibility
    
    def format(self, input_dict):
        return {
                "prompt": self.template.format(**input_dict),
                **input_dict   # keep original keys
            }



In [56]:
class FakeStrOutputParser(Runnable):
    def __init__(self):
        print('String Output Parser Created')

    def invoke(self, llm_output):
        return llm_output['response']
         
    # keep the parse method for compatibility
    
    def parse(self, llm_output):
        return llm_output['response']

In [58]:
class RunnableConnector(Runnable):
    def __init__(self, runnable_list):
        self.runnable_list = runnable_list
        print('Runnable Connector Created')

    def invoke(self, input_data):
        for runnable in self.runnable_list:
            input_data = runnable.invoke(input_data)
        return input_data

In [59]:
template=FakePromptTemplate(
    template='Write a {length} poem about {topic}.',
    input_variables=['length', 'topic']
)

Prompt Template Created


In [60]:
llm=FakeLLM()

LLM Created


In [61]:
parser=FakeStrOutputParser()

String Output Parser Created


In [62]:
chain=RunnableConnector([template, llm, parser])

Runnable Connector Created


In [65]:
chain.invoke({'length':'long', 'topic':'Bangladesh'})
chain.invoke({'length':'short', 'topic':'AI'})


'AI stands for Artificial Intelligence.'